# Accelerometer Log Visualizer
I programmed an Arduino to collect accelerometer data and dump into an SD card as a CSV.  The file format is

(sec since on), (x_last_reading), (y_last_reading), (z_last_reading), (x_reading), (y_reading), (z_reading)

The data only gets logged if last reading is past a certain threshold from first reading

## Generate some random data to test log visualizer

In [ ]:
if False:
    import csv
    import random
    tsec = 0;
    with open('tmp.csv', 'w') as csvfile:
        for x in range(0,100):
            tsec = random.randint(tsec, tsec+3600)
            if tsec >= 60*60*8:
                break;
            spamwriter = csv.writer(csvfile)
            spamwriter.writerow([tsec,
                                 random.randint(50,110),
                                 random.randint(50,110),
                                 random.randint(50,110),
                                 random.randint(50,110),
                                 random.randint(50,110),
                                 random.randint(50,110)])

## Read the log data
Read in CSV and store as magnitude of tuple-2 of (hour in float, magnitude of change)

In [ ]:
import csv
import numpy as np

def read_sleep_data(fn):
    '''Return an array of tuple [(hr_offset, magnitude of movement), (hr_offset, mag)...]'''
    data_log = []
    start_hr = 0 #21.15 # 9:15pm
    ignore_first_hours = 0 #1.5
    with open(fn, 'r') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            sec = long(row[0])
            last = np.array(map(float, row[1:4]))
            cur = np.array(map(float, row[4:]))
            hr = float(sec)/3600.0 
            diff_vec = last-cur
            mag = np.linalg.norm(diff_vec)        
            data_log.append((hr,mag))
        data_log = np.array(data_log)
    
    # Is the first entry larger than 2nd?  If so, we have our start time
    # in the first entry
    # IE 85500,0.1,0,0
    if data_log[0][0] > data_log[1][0]:
        # When did log first start in real time - ie 22.5 == 10:30pm
        start_hr = data_log[0][0]
        ignore_first_hours = data_log[0][1]
        data_log = data_log[1:]
    
    # If second entry is negative than we have the subjective quality encoded 
    # in the magnitude
    if data_log[0][0] < 0:
        quality = data_log[0][1]
        data_log = data_log[1:]
        
    return [start_hr, data_log]

fn = "/Users/jpien/Desktop/datalog_2015_05_20__22_00__23_30.csv"
fn = "/Users/jpien/Desktop/datalog.csv"

[start_hr, data_log] = read_sleep_data(fn)

## Combine the magnitudes of readings within the same time quanta

In [ ]:
def clean_sleep_data(data_log):
    '''Returns cleaned data_log'''
    # Find last second
    [nr, nc] = np.shape(data_log)

    #print data_log
    accum_row = [data_log[0,0], data_log[0,1]]
    new_data_log = [np.copy(accum_row)] # We want first sample
    for row in data_log:
        cur_hr = row[0]
        movement = row[1]
    
        # Time from last time is less than 1.5 seconds away
        if np.abs(cur_hr - accum_row[0]) < 0.003:
            # Sum the movement
            accum_row[1] += movement
        else:
            # New entry
            new_data_log.append(accum_row)
        
            # Start new entry    
            accum_row = [cur_hr, movement]
        
    return np.array(new_data_log)

cleaned_data_log = clean_sleep_data(data_log)

## Compute some data from readings

In [ ]:
def convert_to_readable_time(float_hour):
    am_pm = "am"
    rt_hour = float_hour
    
    if ((int(float_hour / 12)) % 2) == 0:
        am_pm = "am"
    else:
        am_pm = "pm"
    rt_hour = int(float_hour % 12)
        
    the_minute = int((float_hour - int(float_hour)) * 60)
    return str(rt_hour) + ":" + str(the_minute) + am_pm

def analyze_sleep_data_log(data_log):
    # When did we go to sleep - figure out first chunk of quiescent 20 minutes
    last_data = data_log[0]
    sleep_start_hr_offset = 0;
    for i in range(len(data_log)):
        ddd = data_log[i]
        if ddd[0] - last_data[0] > 0.33:
            sleep_start_hr_offset = last_data[0]
            break;
        last_data = ddd
        
    # When did we get up?
    sleep_stop_hr_offset = data_log[-1,0]
    
    # Let's clean up data further by removing small movements
    move_thresh = 10
    new_data_log = []
    for ddd in data_log:
        if ddd[1] > move_thresh:
            new_data_log.append(ddd)
    new_data_log = np.array(new_data_log)
    
    # Between sleep time and get up time, get all the tuples of quiescent chunks
    # Quiescent == minimal movement for 30 min or 0.5 hours
    quiet_intervals = [] # [[(hr,mv),(hr.mv)], [(hr,mv),(hr,mv)], ...]
    last_data = new_data_log[0]
    for i in range(len(new_data_log)):
        ddd = new_data_log[i]
        
        # Before we went to sleep or after we got up
        if ddd[0] < sleep_start_hr_offset:
            last_data = ddd
            continue
            
        if ddd[0] >= sleep_stop_hr_offset:
            break
            
        if ddd[0] - last_data[0] >= 1.0:
            quiet_intervals.append([last_data,ddd])
            
        last_data = ddd
        
    quiet_intervals = np.array(quiet_intervals)
    
    # Average quiet interval
    quiet_hr_array = []
    for itv in quiet_intervals:
        quiet_hrs = itv[1][0] - itv[0][0]
        quiet_hr_array.append(quiet_hrs)
    quiet_hr_array = np.array(quiet_hr_array)
    quiet_hr_intervals = np.shape(quiet_hr_array)[0]
    quiet_hours_mean = quiet_hr_array.mean()
    quiet_hours_std = quiet_hr_array.std()
    #print (quiet_intervals + start_hr) %24
    

    return [sleep_start_hr_offset, sleep_stop_hr_offset, quiet_hours_mean, quiet_hours_std, quiet_hr_intervals]

[ sleep_start, sleep_stop, quiet_hrs_mean, quiet_hrs_std, quiet_hr_intervals ] = analyze_sleep_data_log(cleaned_data_log)

log_start_time = convert_to_readable_time(start_hr)
got_up_time = convert_to_readable_time(start_hr + sleep_stop)
slept_time = convert_to_readable_time(start_hr + sleep_start)

print "We started logging at: " + log_start_time
print "We went to sleep at: " + slept_time
print "We slept for (hours): " + str(sleep_stop - sleep_start)
print "We got up at: " + got_up_time
print "Sleeps cycles with more than 60 min of sleep: " + str(quiet_hr_intervals)
print "Length of sleep cycle fell btw(hours): " + \
    "{0:.2f}".format(quiet_hrs_mean-quiet_hrs_std) + " to " + "{0:.2f}".format(quiet_hrs_mean+quiet_hrs_std)
    #("%.2d" % quiet_hrs_mean-quiet_hrs_std) + " to " + ("%.2d" %quiet_hrs_mean+quiet_hrs_std)
#print "Standard deviation of sleep cycle (hours): " + str(quiet_hrs_std)

## Ignore data from before we went to sleep


In [ ]:
def purge_awake_data(data_log):
    '''Get rid of data from when we determined to be awake'''
    new_data_log = [[0,0]]
    for ddd in data_log:
        if ddd[0] >= sleep_start and \
            ddd[0] < sleep_stop:
            new_data_log.append(ddd)
    return np.array(new_data_log)

plot_data_log = purge_awake_data(cleaned_data_log)

## Plot the data plots to see when we made movements

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
#plt.plot(data_log[:,0],data_log[:,1])
#plt.show()
max_hr = np.max(plot_data_log[:,0]) + 1
fig = plt.figure()
fig.set_size_inches(14, 8)
plt.xticks(np.arange(0,max_hr), np.arange(start_hr,start_hr+max_hr)%24)
plt.bar(plot_data_log[5:,0],plot_data_log[5:,1], width=0.01)
plt.xlabel("Hour", fontsize = 20)
plt.ylabel("Magnitude of movement", fontsize = 20)
plt.show()
